[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abigailhaddad/fedscope_new/blob/main/reconciliation_analysis.ipynb)

# Federal Workforce Data: Do the Numbers Add Up?

OPM publishes two types of federal workforce data:

1. **Headcount snapshots** (Employment data): Monthly counts of how many people work at each agency
2. **Activity records** (Accessions & Separations): Records of people being hired and people leaving

**If the data were perfect, these should match:**
> Change in headcount = New hires − Departures

This notebook finds agencies where these two measures show different numbers. Some difference is expected since the datasets measure slightly different things — we're just flagging larger differences so you can take a closer look.

### Column guide

| Column | What it means |
|--------|---------------|
| `headcount_change` | How much the employee count changed (ending − starting) |
| `hires_minus_departures` | What the change would be based on hiring/leaving records |
| `headcount_says` | Growing or shrinking according to headcount? |
| `activity_says` | Growing or shrinking according to hires/departures? |
| `opposite_stories` | TRUE when headcount and activity point in opposite directions |

In [ ]:
import duckdb
import ipywidgets as widgets
from IPython.display import display

HF_USERNAME = "abigailhaddad"

def parquet_url(data_type: str, month: str) -> str:
    """HuggingFace parquet URL for a dataset."""
    return f"hf://datasets/{HF_USERNAME}/opm-federal-{data_type}-{month}/data.parquet"

## Settings

Use these controls to adjust the analysis:

- **Window (months)**: How many months to look at together (e.g., 4 = compare Jan-Apr headcount change vs Jan-Apr hiring/leaving)
- **Threshold %**: Only flag agencies where the unexplained difference is more than X% of their workforce
- **Min employees**: Ignore tiny agencies with fewer than X employees (they can have big % swings from small changes)
- **Only show opposite stories**: Focus on the most suspicious cases — where headcount says "growing" but activity says "shrinking" (or vice versa)

In [ ]:
MONTHS = ["202501", "202502", "202503", "202504", "202505", "202506", "202507", "202508", "202509"]

# Interactive controls
window_size_widget = widgets.IntSlider(value=4, min=2, max=6, step=1, description='Window (months):')
threshold_pct_widget = widgets.FloatSlider(value=20.0, min=5.0, max=50.0, step=5.0, description='Threshold %:')
min_employees_widget = widgets.IntSlider(value=0, min=0, max=1000, step=50, description='Min employees:')
mismatch_only_widget = widgets.Checkbox(value=False, description='Only show opposite stories')

display(window_size_widget, threshold_pct_widget, min_employees_widget, mismatch_only_widget)

## Load Data

We load three datasets from HuggingFace (originally from [OPM FedScope](https://www.opm.gov/data/)):

- **Employment**: Monthly headcount snapshots — how many people work at each agency
- **Accessions**: New hires — people joining federal service  
- **Separations**: Departures — people leaving federal service (retirement, resignation, termination, etc.)

For accessions and separations, we filter by `personnel_action_effective_date_yyyymm` to use the actual date the action happened, not just when it was recorded.

In [ ]:
def union_all_months(data_type: str, months: list[str]) -> str:
    """Generate SQL to UNION ALL parquet files for all months."""
    selects = [
        f"SELECT *, '{m}' as file_month FROM read_parquet('{parquet_url(data_type, m)}')"
        for m in months
    ]
    return " UNION ALL ".join(selects)


con = duckdb.connect()

# Load all data into views
con.execute(f"CREATE VIEW employment AS {union_all_months('employment', MONTHS)}")
con.execute(f"CREATE VIEW accessions AS {union_all_months('accessions', MONTHS)}")
con.execute(f"CREATE VIEW separations AS {union_all_months('separations', MONTHS)}")

print(f"Loaded {len(MONTHS)} months of data")

## How the Analysis Works

For each agency/subagency, we look at rolling time windows (e.g., Jan-Apr, Feb-May, Mar-Jun...) and calculate:

1. **What did the headcount data show?**  
   `headcount_change = ending_employees − starting_employees`

2. **What did the activity data show?**  
   `hires_minus_departures = total_hires − total_departures`

3. **What's the difference?**  
   `unexplained_diff = headcount_change − hires_minus_departures`

These datasets measure slightly different things, so some difference is normal. We're just flagging cases where the difference is larger than usual so you can take a closer look.

In [ ]:
def get_windows(months: list[str], window_size: int) -> list[tuple[str, str]]:
    """Generate (start_month, end_month) tuples for rolling windows."""
    return [(months[i], months[i + window_size - 1]) 
            for i in range(len(months) - window_size + 1)]


def window_discrepancy_sql(start_month: str, end_month: str, months: list[str]) -> str:
    """SQL to compute differences for a window, by subagency."""
    window_months = months[months.index(start_month):months.index(end_month) + 1]
    month_list = ", ".join(f"'{m}'" for m in window_months)
    
    return f"""
    WITH start_emp AS (
        SELECT agency, agency_code, agency_subelement, agency_subelement_code,
               SUM(CAST(count AS INTEGER)) as emp
        FROM employment WHERE file_month = '{start_month}'
        GROUP BY ALL
    ),
    end_emp AS (
        SELECT agency, agency_code, agency_subelement, agency_subelement_code,
               SUM(CAST(count AS INTEGER)) as emp
        FROM employment WHERE file_month = '{end_month}'
        GROUP BY ALL
    ),
    acc AS (
        SELECT agency, agency_code, agency_subelement, agency_subelement_code,
               SUM(CAST(count AS INTEGER)) as total
        FROM accessions 
        WHERE personnel_action_effective_date_yyyymm IN ({month_list})
        GROUP BY ALL
    ),
    sep AS (
        SELECT agency, agency_code, agency_subelement, agency_subelement_code,
               SUM(CAST(count AS INTEGER)) as total
        FROM separations
        WHERE personnel_action_effective_date_yyyymm IN ({month_list})
        GROUP BY ALL
    )
    SELECT 
        COALESCE(s.agency, e.agency) as agency,
        COALESCE(s.agency_code, e.agency_code) as agency_code,
        COALESCE(s.agency_subelement, e.agency_subelement) as agency_subelement,
        COALESCE(s.agency_subelement_code, e.agency_subelement_code) as agency_subelement_code,
        '{start_month}' as window_start,
        '{end_month}' as window_end,
        COALESCE(s.emp, 0) as starting_employees,
        COALESCE(e.emp, 0) as ending_employees,
        COALESCE(a.total, 0) as hires,
        COALESCE(sp.total, 0) as departures,
        (COALESCE(e.emp, 0) - COALESCE(s.emp, 0)) as headcount_change,
        (COALESCE(a.total, 0) - COALESCE(sp.total, 0)) as hires_minus_departures,
        (COALESCE(e.emp, 0) - COALESCE(s.emp, 0)) - (COALESCE(a.total, 0) - COALESCE(sp.total, 0)) as unexplained_diff,
        CASE WHEN COALESCE(s.emp, 0) > 0 
             THEN ABS((COALESCE(e.emp, 0) - COALESCE(s.emp, 0)) - (COALESCE(a.total, 0) - COALESCE(sp.total, 0))) * 100.0 / s.emp
             ELSE NULL END as unexplained_diff_pct
    FROM start_emp s
    FULL OUTER JOIN end_emp e USING (agency, agency_code, agency_subelement, agency_subelement_code)
    LEFT JOIN acc a USING (agency, agency_code, agency_subelement, agency_subelement_code)
    LEFT JOIN sep sp USING (agency, agency_code, agency_subelement, agency_subelement_code)
    """

In [ ]:
def run_analysis(window_size: int, threshold_pct: float, min_employees: int = 0):
    """Run the full analysis with given parameters."""
    windows = get_windows(MONTHS, window_size)
    print(f"Settings: {window_size}-month windows, >{threshold_pct}% threshold, min {min_employees} employees")
    
    con.execute("DROP VIEW IF EXISTS all_windows")
    all_windows_sql = " UNION ALL ".join(
        f"({window_discrepancy_sql(start, end, MONTHS)})" 
        for start, end in windows
    )
    con.execute(f"CREATE VIEW all_windows AS {all_windows_sql}")
    
    result = con.execute(f"""
        WITH flagged AS (
            SELECT * FROM all_windows 
            WHERE unexplained_diff_pct > {threshold_pct} AND starting_employees >= {min_employees}
        ),
        ordered AS (
            SELECT *, LAG(window_end) OVER (PARTITION BY agency_code, agency_subelement_code ORDER BY window_start) as prev_end
            FROM flagged
        ),
        islands AS (
            SELECT *,
                   SUM(CASE WHEN prev_end IS NULL OR window_start > prev_end THEN 1 ELSE 0 END) 
                       OVER (PARTITION BY agency_code, agency_subelement_code ORDER BY window_start) as island_id
            FROM ordered
        ),
        merged_islands AS (
            SELECT agency, agency_code, agency_subelement, agency_subelement_code, island_id,
                   MIN(window_start) as range_start, MAX(window_end) as range_end,
                   COUNT(*) as windows_in_island,
                   SUM(headcount_change) as total_headcount_change,
                   SUM(hires_minus_departures) as total_hires_minus_departures,
                   AVG(starting_employees) as avg_employees
            FROM islands
            GROUP BY agency, agency_code, agency_subelement, agency_subelement_code, island_id
        )
        SELECT agency, agency_code, agency_subelement, agency_subelement_code,
               STRING_AGG(range_start || '-' || range_end, ', ' ORDER BY range_start) as flagged_periods,
               SUM(windows_in_island)::INT as n_windows,
               ROUND(AVG(avg_employees))::INT as avg_employees,
               SUM(total_headcount_change)::INT as headcount_change,
               SUM(total_hires_minus_departures)::INT as hires_minus_departures,
               CASE WHEN SUM(total_headcount_change) > 0 THEN 'growing' 
                    WHEN SUM(total_headcount_change) < 0 THEN 'shrinking' 
                    ELSE 'flat' END as headcount_says,
               CASE WHEN SUM(total_hires_minus_departures) > 0 THEN 'growing' 
                    WHEN SUM(total_hires_minus_departures) < 0 THEN 'shrinking' 
                    ELSE 'flat' END as activity_says,
               CASE WHEN (SUM(total_headcount_change) > 0 AND SUM(total_hires_minus_departures) < 0) 
                      OR (SUM(total_headcount_change) < 0 AND SUM(total_hires_minus_departures) > 0) 
                    THEN true ELSE false END as opposite_stories
        FROM merged_islands
        GROUP BY agency, agency_code, agency_subelement, agency_subelement_code
        ORDER BY opposite_stories DESC, n_windows DESC, agency, agency_subelement
    """).fetchdf()
    
    n_opposite = result['opposite_stories'].sum()
    print(f"Found {len(result)} subagencies above threshold")
    print(f"  {n_opposite} show opposite directions (headcount vs activity disagree)\n")
    return result


def export_to_csv(df, filename="reconciliation_results.csv"):
    """Export results to CSV."""
    df.to_csv(filename, index=False)
    print(f"Exported {len(df)} rows to {filename}")


def drill_down(agency_code: str, subelement_code: str):
    """Show window-by-window breakdown for a specific subagency."""
    result = con.execute(f"""
        SELECT window_start, window_end, 
               starting_employees, ending_employees, 
               hires, departures, 
               headcount_change, hires_minus_departures,
               unexplained_diff, 
               ROUND(unexplained_diff_pct, 1) as unexplained_diff_pct
        FROM all_windows
        WHERE agency_code = '{agency_code}' 
          AND agency_subelement_code = '{subelement_code}'
        ORDER BY window_start
    """).fetchdf()
    
    if len(result) == 0:
        print(f"No data found for {agency_code}/{subelement_code}")
        return None
    
    agency_name = con.execute(f"""
        SELECT DISTINCT agency, agency_subelement 
        FROM all_windows 
        WHERE agency_code = '{agency_code}' AND agency_subelement_code = '{subelement_code}'
    """).fetchone()
    
    print(f"{agency_name[0]} / {agency_name[1]}")
    print(f"Codes: {agency_code} / {subelement_code}\n")
    return result

## Run Analysis

Adjust the sliders above and re-run this cell to see results with different parameters.

In [ ]:
# Run with current widget values
result = run_analysis(
    window_size_widget.value, 
    threshold_pct_widget.value,
    min_employees_widget.value
)

if mismatch_only_widget.value:
    result = result[result['opposite_stories'] == True]
    print(f"Filtered to {len(result)} with opposite stories\n")

result

## Export Results

In [ ]:
# Export current results to CSV
export_to_csv(result, "reconciliation_results.csv")

## Drill-Down

Want to see the details for a specific agency? Use `drill_down(agency_code, subelement_code)` to see the window-by-window numbers.

Find the codes in the `agency_code` and `agency_subelement_code` columns of the results above.

In [ ]:
# Example: Look at Interior's Office of the Secretary
# Headcount shows +15,997 but hires-departures shows -470
drill_down("IN", "IN01")